In [ ]:
'''
This script shows how to predict stock prices using a basic RNN
'''
import tensorflow as tf
import numpy as np
import matplotlib
import os

tf.set_random_seed(777)  # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt


def MinMaxScaler(data):
    ''' Min Max Normalization

    Parameters
    ----------
    data : numpy.ndarray
        input data to be normalized
        shape: [Batch size, dimension]

    Returns
    ----------
    data : numpy.ndarry
        normalized data
        shape: [Batch size, dimension]

    References
    ----------
    .. [1] http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

    '''
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)


# train Parameters
timesteps = seq_length = 7
data_dim = 1
hidden_dim = 10
output_dim = 1
learing_rate = 0.01
iterations = 500

# Open, High, Low, Volume, Close
xy = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1]  # reverse order (chronically ordered)
xy = MinMaxScaler(xy)
x = xy[:, [-1]]
y = xy[:, [-1]]  # Close as label

# build a dataset
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    #print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

# train/test split
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learing_rate)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    train_predict = sess.run(Y_pred, feed_dict={X: trainX})
    testX = np.array([trainY[-7:,:]])
    testX_predict = []
    for _ in range(test_size):
        test_predict = sess.run(Y_pred, feed_dict={X: testX})
        testX = np.concatenate([testX[:,1:,:],test_predict.reshape(1,1,1)],axis=1)
        testX_predict.append(test_predict)
    rmse = sess.run(rmse, feed_dict={
                    targets: testY, predictions: np.array(testX_predict).reshape(-1,1)})
    print("RMSE: {}".format(rmse))

    # Plot predictions
    plt.plot(trainY)
    plt.plot(train_predict)
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()
    plt.plot(testY)
    plt.plot(np.array(testX_predict).reshape(-1,1))
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()

[step: 0] loss: 80.8292388916
[step: 1] loss: 57.9562568665
[step: 2] loss: 40.0421791077
[step: 3] loss: 26.5900650024
[step: 4] loss: 17.2489242554
[step: 5] loss: 11.8747959137
[step: 6] loss: 10.2878761292
[step: 7] loss: 11.6552877426
[step: 8] loss: 13.9523906708
[step: 9] loss: 14.9536037445
[step: 10] loss: 13.888710022
[step: 11] loss: 11.4069013596
[step: 12] loss: 8.59345436096
[step: 13] loss: 6.29478740692
[step: 14] loss: 4.90161752701
[step: 15] loss: 4.40206480026
[step: 16] loss: 4.53491401672
[step: 17] loss: 4.94256448746
[step: 18] loss: 5.28714609146
[step: 19] loss: 5.32537889481
[step: 20] loss: 4.94640016556
[step: 21] loss: 4.17888069153
[step: 22] loss: 3.17432808876
[step: 23] loss: 2.17054510117
[step: 24] loss: 1.4323785305
[step: 25] loss: 1.16316497326
[step: 26] loss: 1.39520227909
[step: 27] loss: 1.91753363609
[step: 28] loss: 2.35184860229
[step: 29] loss: 2.40185976028
[step: 30] loss: 2.06402349472
[step: 31] loss: 1.5847928524
[step: 32] loss: 1.24

[step: 261] loss: 0.741783201694
[step: 262] loss: 0.740523338318
[step: 263] loss: 0.73926794529
[step: 264] loss: 0.738017082214
[step: 265] loss: 0.736771345139
[step: 266] loss: 0.73553019762
[step: 267] loss: 0.734294056892
[step: 268] loss: 0.73306286335
[step: 269] loss: 0.731836855412
[step: 270] loss: 0.730616211891
[step: 271] loss: 0.729400873184
[step: 272] loss: 0.728190779686
[step: 273] loss: 0.726985991001
[step: 274] loss: 0.725787103176
[step: 275] loss: 0.724593639374
[step: 276] loss: 0.723405957222
[step: 277] loss: 0.722223937511
[step: 278] loss: 0.721047997475
[step: 279] loss: 0.719877660275
[step: 280] loss: 0.718713462353
[step: 281] loss: 0.717555344105
[step: 282] loss: 0.716403126717
[step: 283] loss: 0.715257108212
[step: 284] loss: 0.714117527008
[step: 285] loss: 0.712983906269
[step: 286] loss: 0.711856663227
[step: 287] loss: 0.710735678673
[step: 288] loss: 0.709621369839
[step: 289] loss: 0.708513081074
[step: 290] loss: 0.707411527634
[step: 291] l

In [2]:
# Functions to show the Graphs

import numpy as np
from IPython.display import clear_output, Image, display, HTML


def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [3]:
show_graph(tf.get_default_graph())

In [5]:
# END